In [2]:
from datetime import date, timedelta
import time
import pandas as pd
import binance
from binance import Client
import sqlalchemy
import pendulum
from ta import add_all_ta_features
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
import uuid
import time, datetime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import joblib



In [3]:
secret_key = os.environ.get('BI_SECRETKEY')
api_key = os.environ.get('BI_APIKEY')
engine = sqlalchemy.create_engine('sqlite:///crypto.db')
client = Client(api_key,secret_key)
log_cols = ['Pairing','Open_Time','Open_Price','Close_Time','Close_Price']
trade_log_df = pd.DataFrame(columns = log_cols)

In [8]:
class coin():
    def __init__(self,ticker,com_ticker,run_through=True):
        self.name = ticker
        self.com_ticker = com_ticker
        self.table_name = f'{self.name}_{self.com_ticker}'
        self.cols =['Open_Time',
                    f'{com_ticker}_Open',
                    f'{com_ticker}_High',
                    f'{com_ticker}_Low',
                    f'{com_ticker}_Close',
                    f'{com_ticker}_Volume',
                    'Close_Time',
                    'Quote_Asset_Volume',
                    'Number_Of_Trades',
                    'TBBAV',
                    'TBQAV',
                    'Ignore']
        self.time_frame = 4
        self.path = f'/Users/bdigio17/Documents/Python Learning/Projects/Stock/{self.name}/'
        self.exists = False
        self.new_data = 0
        #Checks if coin has a directory
        if not os.path.isdir(self.path):
            os.mkdir(self.path)

        #Analsye and model coin
        if run_through:
            self.historical()
            if not self.exists:
                self.technical_analysis()
                self.label()

            self.to_sql()
            self.build_model()


    #Get historical data for analysis
    def historical(self):
        #check if table doesn't exist in SQL db
        if not sqlalchemy.inspect(engine).has_table(self.table_name):
            klines = client.get_historical_klines(symbol=f'{self.name}{self.com_ticker}',
                                interval=client.KLINE_INTERVAL_1HOUR,
                                start_str='Jan 1, 2017',
                                end_str=pendulum.today().to_formatted_date_string())
            
            #Assign Column Names
            df = pd.DataFrame(klines, columns=self.cols)

            #Clean data types
            df.Open_Time = pd.to_datetime(df.Open_Time, unit='ms')
            df.Close_Time = pd.to_datetime(df.Close_Time, unit='ms')
            for colmn in df.columns:
                if not 'Time' in colmn:
                    df[colmn] = df[colmn].astype(float)

            self.data = df
                    
        #Load table 
        else:
            df = pd.read_sql(self.table_name, engine)
            self.data = df
            self.exists = True
            #check for updates

            #Assign last update to string
            final_time = pd.to_datetime(df['Close_Time'].tail(1).values).strftime('%b %d, %Y')[0]
            
            if df['Close_Time'].tail(1).values != pendulum.today().to_formatted_date_string():
                klines = client.get_historical_klines(symbol=f'{self.name}{self.com_ticker}',
                                interval=client.KLINE_INTERVAL_1HOUR,
                                start_str=final_time,
                                end_str=pendulum.today().to_formatted_date_string())
                #Assign Column Names
                df = pd.DataFrame(klines, columns=self.cols)
                self.new_data = len(df)

                #Clean data types
                df.Open_Time = pd.to_datetime(df.Open_Time, unit='ms')
                df.Close_Time = pd.to_datetime(df.Close_Time, unit='ms')
                for colmn in df.columns:
                    if not 'Time' in colmn:
                        df[colmn] = df[colmn].astype(float)
        
                #match tables
                self.df_update(df)
                
        return self.data

    def technical_analysis(self):
        self.data = add_all_ta_features(self.data,
                                        open = f'{self.com_ticker}_Open',
                                        high = f'{self.com_ticker}_High',
                                        low = f'{self.com_ticker}_Low',
                                        close = f'{self.com_ticker}_Close',
                                        volume = f'{self.com_ticker}_Volume',
                                        fillna=True)
        return self.data

    def to_sql(self):
        if not self.exists:
            self.data.to_sql(self.table_name, engine, if_exists='append', index='False')
        else:
            # If it already exists...
            temp_df = self.data.tail(self.new_data)
            temp_df.drop(temp_df.iloc[:,0:1],axis=1,inplace =True)
            temp_df.to_sql(self.table_name, engine, if_exists='append', index='False')

            

    #Creates 'labels' i.e. changes in price that can be used to model
    def label(self):
        labels = []
        open = f'{self.com_ticker}_Open'
        close = f'{self.com_ticker}_Close'

        #Create list of change in open price to close price by the timeframe
        for i in range(len(self.data)-self.time_frame):
            if len(self.data) >= i+self.time_frame:
                change = self.data[close].loc[i+self.time_frame] - self.data[open].loc[i]
                labels.append(change)
        
        #Shorten data by the time frame to avoid nulls
        self.data = self.data.head(len(self.data)-self.time_frame)
        self.data[f'Increase_after_{self.time_frame}'] = labels
        inc_dec = []

        #Checks for increase and provides boolean
        for i in range(len(self.data)):
            if self.data[f'Increase_after_{self.time_frame}'].loc[i] > 0:
                inc_dec.append(1)
            else:
                inc_dec.append(0)
        self.data['Increased?'] = inc_dec
        return self.data

    def df_update(self,df):

        self.data.drop(self.data.loc[:,'volume_adi':'others_cr'],axis=1, inplace=True)
        self.data.append(df,ignore_index=True)
        self.technical_analysis()
        self.label()

    def build_model(self):
        filename = f'{self.path}model_{self.name}_{self.com_ticker}.sav'

         #increase regression model
        self.return_model = LinearRegression()
        return_x = self.data.loc[:,'volume_adi':'others_cr'].values
        return_y = self.data.loc[:,f'{self.com_ticker}_Close']
        self.return_model.fit(return_x,return_y)

        #Check if model exists
        if not os.path.isfile(filename):
            x = self.data.loc[:,'volume_adi':'others_cr'].values
            y = self.data[f'Increased?'].values
            x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.20, random_state=1)
            models = []
            models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
            models.append(('LDA', LinearDiscriminantAnalysis()))
            models.append(('KNN', KNeighborsClassifier()))
            models.append(('NB', GaussianNB()))

            #Evaluate each model in turn for price change
            results = []
            names = []
            average_result = []
            for name, model in models:
                kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
                cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring='accuracy')
                results.append(cv_results)
                names.append(name)
                average_result.append(cv_results.mean())
                print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

            #Choose model
            best_result = max(average_result)
            self.model = models[average_result.index(best_result)][1]
            self.model.fit(x_train, y_train)
            predictions = self.model.predict(x_validation)
            
            #Evaluate predictions
            #print(accuracy_score(y_validation, predictions))
            #print(confusion_matrix(y_validation, predictions))
            #print(classification_report(y_validation, predictions))

            #Saves model
            joblib.dump(self.model, filename)

           



        else:
            #Loads model
            self.model = joblib.load(filename)

    def reload(self,build = False): #for test
        self.historical()
        if not self.exists:
            self.technical_analysis()
            self.label()

        self.to_sql()
        if build:
            self.build_model()

    def predict(self,x):
        prediction = self.model.predict(x)
        return prediction

    def predict_inc(self,x):
        prediction = self.return_model.predict(x)
        return prediction

In [11]:
test = coin('BTC','USDT')

/var/folders/r6/7nfjjkyd1p9_3r4gjj6f8zqx0rynt6/T/ipykernel_54767/3726806125.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if df['Close_Time'].tail(1).values != pendulum.today().to_formatted_date_string():
/var/folders/r6/7nfjjkyd1p9_3r4gjj6f8zqx0rynt6/T/ipykernel_54767/3726806125.py:139: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data.append(df,ignore_index=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/var/folders/r6/7nfjjkyd1p9_3r4gjj6f8zqx0rynt6

In [9]:

def trade_decision(coin): 
    global acc_bal  #for test
    stop_loss = 0.15 #stop loss %
    day_trade_no = 1 #number of trades/day
    #acc_bal = [float(client.get_asset_balance(asset=coin.name)['free']),float(client.get_asset_balance(asset=coin.com_ticker)['free'])]

    while True:
    #Determine volume of trade (40% of balance)
        if acc_bal != 0:
            trade_vol = acc_bal[1] * 0.40
        else:
            print(f'{coin.com_ticker} balance is 0')
        trade_open = False

        trade_fee = 0.001

        #refresh sql
        df = pd.read_sql(f'{coin.name}_{coin.com_ticker}', engine)
        x = df.tail(1)
        x = x.loc[:,'volume_adi':'others_cr'].values
        prediction = coin.predict(x)
        decision=0
        
        while prediction > 0.5:
            decision=0
            
            #calculate trade fees
            total_fee = trade_vol * trade_fee

            return_prediction = coin.predict_inc(x)
            return_prediction = float(return_prediction[0])
            profit_prediction = (trade_vol/return_prediction)*(return_prediction - df.tail(1).loc[:,f'{coin.com_ticker}_Open'])
            temp_price = df.tail(1).loc[:,f'{coin.com_ticker}_Open']
            temp_price = temp_price.values
            profit_prediction = profit_prediction.values

            #Trade if higher
            if profit_prediction > total_fee and trade_open == False:
                print(f'Position predicted to increase, trading {trade_vol} of {coin.name}_{coin.com_ticker} at {temp_price} for a predicted profit of {profit_prediction}')
                decision=1
                trade_open = execute_trade(coin,decision,df,trade_open,trade_vol)

            elif profit_prediction < total_fee and trade_open:
                print(f'Position predicted to increase, but fees exceed expected increase, waiting')

            #reset and retest
            print('Waiting for next hour to retest')
            time.sleep(60 * (60 - datetime.datetime.now().minute))
            coin.reload()
            #reload sql
            df = pd.read_sql(f'{coin.name}_{coin.com_ticker}', engine)
            x = df.tail(1)
            x = x.loc[:,'volume_adi':'others_cr'].values
            prediction = coin.predict(x)
 
        if trade_open and prediction == 0:
                print('Position not precicted to increase, selling current position')
                decision=2
                trade_open = execute_trade(coin,decision,df,trade_open,trade_vol)

        elif not trade_open:
            print('No trade open and no expected increase')
            
        print('Waiting until next hour to retest')    
        time.sleep(60 * (60 - datetime.datetime.now().minute))
        coin.reload() #for test


def execute_trade(coin, decision, df, trade_open,trade_vol): #decision 0 = no trade, 1 = buy, 2 = sell
    global trade_no
    global acc_bal #for test
    print('trade')
    if decision == 1:
        #send trade request
        #Only 1 trade at a time as of now so basic updating
        temp = [f'{coin.name}_{coin.com_ticker}',df.tail(1).loc[:,'Open_Time'],df.tail(1).loc[:,f'{coin.com_ticker}_Open'],0,0]
        trade_log_df.append(temp)

        trade_open = True

        acc_bal[1] = acc_bal[1] - trade_vol

    elif decision == 2:
    #After closing
        trade_log_df['Close_Time'].iloc[trade_no] = df.tail(1).loc[:,'Close_Time']
        trade_log_df['Close_Price'].iloc[trade_no] = df.tail(1).loc[:,f'{coin.com_ticker}_Close']

        trade_open = False
        realised_profit =  (trade_log_df['Open_Price'].iloc[trade_no] - trade_log_df['Close_Price'].iloc[trade_no]) * (trade_vol / trade_log_df['Open_Price'].iloc[trade_no])
        acc_bal[1] = acc_bal[1] + realised_profit
        print(f'${realised_profit} of profit realised')
        trade_no += 1
        print(acc_bal)
    return trade_open


In [12]:
global acc_bal
acc_bal = [0.004,1000]
global trade_no
trade_no = 0
while acc_bal[1] > 0:
    trade_decision(test)
    



No trade open and no expected increase
Waiting until next hour to retest


/var/folders/r6/7nfjjkyd1p9_3r4gjj6f8zqx0rynt6/T/ipykernel_54767/3726806125.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if df['Close_Time'].tail(1).values != pendulum.today().to_formatted_date_string():


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))